# SCIENTIFIC INVESTIGATION ON TITANIC'S CASE

## 01. IMPORTING DATA

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid') # Plots are gonna have white grids.

## 02. LOADING THE DATASET

In [ ]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
combined_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True) 

## 03. DATASET SUMMARY

In [ ]:
print("=" * 50)
print("TITANIC DISASTER DATASET SUMMARY")
print("=" * 50)

train_male = (train_df["Sex"] == "male").sum()
train_female = (train_df["Sex"] == "female").sum()

print(f"Trainning Set:  {train_df.shape[0]} Passengers ({train_male}M and {train_female}F) & {train_df.shape[1]} Features.")
print(f"Testing Set:    {test_df.shape[0]} Passengers & {test_df.shape[1]} Features.")
print(f"Combined Set:  {combined_df.shape[0]} Passengers & {combined_df.shape[1]} Features.")

print("\nFeatures:")
for i, feature in enumerate(combined_df.columns, start = 1):
    print(f"   {i:02d}. {feature}")
print("=" * 50)

## 04. WHAT STORY DOES THE DATA TELL US?

In [ ]:
# Creating a 3-axes figure.
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. MISSING VALUES HEATMAP
ax1 = axes[0]
missing_data = train_df.isnull().sum().sort_values(ascending=False)
missing_pct = ( missing_data / len(train_df) * 100 ).round(2)

missing_df = pd.DataFrame({
    "Missing": missing_data,
    "Percentage": missing_pct                  
})
missing_df = missing_df[missing_df["Missing"] > 0]

colors = [
    "#e94560" if x > 50
    else
        "#533483" if x > 10
        else "#0f3460"
    for x in missing_df["Percentage"]
] # BLUE for 0 < x <= 10; PURPLE for 10 < x <= 50; and PINK for x >= 50.

bars = ax1.barh(missing_df.index, missing_df["Percentage"], color = colors)
ax1.set_title("Missing Values x Features", fontweight="bold")
ax1.set_xlabel("Percentage (%)")
ax1.bar_label(bars, fmt="%.1f%%", padding=5)
ax1.set_xlim(0, 100)

# 2. SURVIVAL DISTRIBUTION
ax2 = axes[1]
survival_counts = train_df['Survived'].value_counts()

colors_surv = ['#e94560', '#16213e']
wedges, texts, autotexts = ax2.pie(survival_counts, labels=["Died", "Survived"], autopct='%.1f%%', explode=(0.02, 0.02), colors=colors_surv)

ax2.set_title("Overall Survival Rate", fontweight="bold")
autotexts[1].set_color("white")
autotexts[0].set_fontweight("bold")
autotexts[1].set_fontweight("bold")

# 3. AGE DISTRIBUTION
ax3 = axes[2]
ax3.hist(train_df["Age"], bins=30, color="#0f3460", edgecolor="white", alpha=0.8)
ax3.axvline(train_df["Age"].median(), color="red", linestyle="--", linewidth=1.5, label = f"Median: {train_df['Age'].median():.0f}")
ax3.set_title("Age Distribution", fontweight="bold")
ax3.legend()

In [ ]:
print("=" * 60)
print("QUICK STATISTICS")
print("=" * 60)
print(f"Survival Rate:    { ((train_df['Survived'] == 1).sum() / len(train_df) * 100):.2f}%")
print(f"Average Age:      { (train_df['Age'].sum() / len(train_df)):.2f}")
print(f"Average Fare:    ${ (train_df['Fare'].sum() / len(train_df)):.2f}")
print(f"Avg Family Size:  { ( train_df['SibSp'] + train_df['Parch'] + 1 ).sum() / len(train_df):.1f}")

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

colors = ["#e94560", "#533483", "#0f3460"]

# 1. SURVIVAL BY SEX
ax1 = axes[0, 0]
survival_by_sex = train_df.groupby("Sex")["Survived"].mean() * 100

bars = ax1.bar(["Female", "Male"], survival_by_sex.values, color = colors)
ax1.bar_label(bars, fmt = "%.1f%%", padding = 5)
ax1.set_title("Survival by Sex", fontweight = "bold")
ax1.set_ylim(0, 100)
#ax1.set_xlabel("Sex")
ax1.set_ylabel("Survival Rate (%)")

# 2. SURVIVAL BY PASSENGER CLASS
ax2 = axes[0, 1]
survival_by_pclass = train_df.groupby("Pclass")["Survived"].mean() * 100

bars = ax2.bar(["1st Class", "2nd Class", "3rd Class"], survival_by_pclass.values, color = colors)
ax2.bar_label(bars, fmt = "%.1f%%", padding = 5)
ax2.set_title("Survival by Class", fontweight = "bold")
ax2.set_ylim(0, 70)
#ax2.set_xlabel("Classes")
ax2.set_ylabel("Survival Rate (%)")

# 3. SURVIVED BY EMBARKED
ax3 = axes[0, 2]
survival_by_embarked = train_df.groupby("Embarked")["Survived"].mean() * 100

bars = ax3.bar(["Cherbourg", "Queenstown", "Southampton"], survival_by_embarked.values, color = colors)
ax3.bar_label(bars, fmt = "%.1f%%", padding = 5)
ax3.set_title("Survived by Embarkation Port", fontweight = "bold")
ax3.set_ylabel("Survival Rate (%)")
ax3.set_ylim(0, 65)

# 4. SURVIVAL BY AGE (KDE DISTRIBUTION)
ax4 = axes[1, 0]
# survival_by_age = train_df.groupby("Age")["Survived"].mean() * 100

train_df[ train_df["Survived"] == 1 ]["Age"].dropna().plot(kind = "kde", ax = ax4, label = "Survived", color = "orange")
ax4.set_title("Age Distribution by Survival", fontweight = "bold")

train_df[ train_df["Survived"] == 0 ]["Age"].dropna().plot(kind = "kde", ax = ax4, label = "Died", color = "#533483")
ax4.set_title("Age Distribution by Survival", fontweight = "bold")

ax4.legend()
ax4.set_xlabel("Age")
ax4.set_xlim(0, 80)
ax4.set_ylim(0, 0.033)

# 5. FARE DISTRIBUTION BY SURVIVAL (BOXPLOT DISTRIBUTION)
ax5 = axes[1, 1]

train_df.boxplot( column = "Fare", by = "Survived",
                  ax = ax5, patch_artist=True,
                  boxprops = dict(facecolor = "#0f3460"),
                  medianprops = dict(color = "red")
                )
ax5.set_title("Fare Distribution by Survival", fontweight = "bold")
ax5.set_ylabel("Fare ($)")
ax5.set_xticklabels(["Died", "Survived"])
ax5.set_ylim(-0.1, 300)

# 6. SURVIVAL BY FAMILY SIZE
ax6 = axes[1, 2]
survival_by_fsize = train_df.groupby( train_df["SibSp"] + train_df["Parch"] + 1 )["Survived"].mean() * 100

bars = ax6.bar(survival_by_fsize.index, survival_by_fsize.values, color = "#0f3460")
ax6.bar_label(bars, fmt = "%.1f%%", padding = 5)
ax6.set_title("Survival by Family Size", fontweight = "bold")
ax6.set_xlabel("Family Size")
ax6.set_ylabel("Survival Rate (%)")
ax6.set_xlim(0, 7.99)
ax6.set_ylim(0, 80)

In [ ]:
survival_by_pclass = train_df.groupby("Pclass")["Survived"].mean() * 100
survival_by_pclass

In [ ]:
train_df[ train_df["Embarked"].isnull() ]

In [ ]:
survival_by_embarked = train_df.groupby("Embarked")["Survived"].sum()
survival_by_embarked

In [ ]:
size = train_df["SibSp"] + train_df["Parch"] + 1
size.values